<a href="https://colab.research.google.com/github/monsterhunters/Lora-Training-GUI/blob/main/Lora_Training_GUI_V1_2414.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kohya SS GUI Based Ver 24.1.1
---

📌 **This notebook has been updated [here](https://github.com/monsterhunters/Lora-Training-GUI.git)!**

🖋 **Author**: [kyawkaung](https://colab.research.google.com/github/monsterhunters/Lora-Training-GUI/blob/main/Lora_Training_GUI_V1_2411.ipynb)

😎 **Support the Project**:

If you find this project useful, please consider supporting it:

<a href="https://ko-fi.com/kyawkaung" target="_blank">
    <img src="https://storage.ko-fi.com/cdn/kofi2.png?v=3" alt="Buy Me a Coffee at ko-fi.com" height="36">
</a>

---

In [ ]:
#@title # Install 🚀
#@markdown Kohya SS GUI by Codemaster
mount_gdrive = True # @param {type:"boolean"}


import os, subprocess, time, glob, zipfile, shutil
from IPython.display import clear_output, display, HTML
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from urllib.parse import unquote
from google.colab.output import eval_js
import pkg_resources
import time
try:
  start_colab
except:
  start_colab = int(time.time())-5

start_install = int(time.time())

print('Info - The whole process may take up to 10 min. So, please be patient...')

def mountGdrive():
    print('Mounting Gdrive')
    from google.colab import drive
    drive.mount('/content/drive')

if mount_gdrive:
    mountGdrive()

print('Installing torch & xformers')
with capture.capture_output() as cap:
    !pip3 uninstall -y torch torchvision torchaudio triton
    !pip install -q torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118
    !pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.23.post1%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl#sha256=dc5f828dbe187c1bf69d41853a55170d2506ff4c40fc0dfbea3bc7e18daed2e5

print('Installing dependencies')
with capture.capture_output() as cap:
    %cd /content
    !git clone -b v24.1.4 https://github.com/bmaltais/kohya_ss.git
    %cd /content/kohya_ss
    !pip install -q accelerate==0.25.0 aiofiles==23.2.1 altair==4.2.2 dadaptation==3.1 diffusers[torch]==0.25.0 easygui==0.98.3
    !pip install -q einops==0.7.0 fairscale==0.4.13 ftfy==6.1.1 gradio==4.26.0 huggingface-hub==0.20.1 imagesize==1.4.1
    !pip install -q invisible-watermark==0.2.0 lion-pytorch==0.0.6 lycoris_lora==2.2.0.post3 omegaconf==2.3.0 onnx==1.15.0
    !pip install -q prodigyopt==1.0 protobuf==3.20.3 open-clip-torch==2.20.0 opencv-python==4.7.0.68 prodigyopt==1.0
    !pip install -q pytorch-lightning==1.9.0 rich>=13.7.1 safetensors==0.4.2 scipy==1.11.4 timm==0.6.12 tk==0.1.0 toml==0.10.2
    !pip install -q transformers==4.38.0 voluptuous==0.13.1 wandb==0.15.11 scipy==1.11.4
    !pip install -q tensorboard==2.15.2 tensorflow==2.15.0.post1 onnxruntime-gpu==1.17.1

print('Setting Cuda')
with capture.capture_output() as cap:
    %cd /content
    !git clone -b 0.41.0 https://github.com/TimDettmers/bitsandbytes
    %cd /content/bitsandbytes
    !CUDA_VERSION=118 make cuda11x
    !python setup.py install
    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
    %env LD_PRELOAD=/content/libtcmalloc_minimal.so.
    os.environ["SAFETENSORS_FAST_GPU"]='1'
    os.environ["CUDA_MODULE_LOADING"]="LAZY"
    os.environ["colab_url"] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["PYTHONWARNINGS"] = "ignore"
    # %env PYTHONDONTWRITEBYTECODE=1
    os.environ["TCMALLOC_AGGRESSIVE_DECOMMIT"] = "t"

print('Almost finish....')
with capture.capture_output() as cap:
    %cd /content/kohya_ss
    !git clone https://github.com/kohya-ss/sd-scripts.git
    !wget https://raw.githubusercontent.com/monsterhunters/stablecode/master/cf.py
    !python cf.py
    !pip install -e ./sd-scripts
    #!git clone https://github.com/kohya-ss/sd-scripts.git

install_time = timedelta(seconds=time.time()-start_install)
print("\r🚀 Finished unpacking. Took","%02d:%02d:%02d ⚡\n" % (install_time.seconds / 3600, (install_time.seconds / 60) % 60, install_time.seconds % 60), end='', flush=True)




In [ ]:
#@title # Run 🚀
import subprocess
import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nUI finished loading, trying to launch (if it gets stuck here, it is having issues)\n")

    p = subprocess.Popen(["wex", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("This is the URL to access UI:", l[l.find("http"):], end='')

# Start the iframe_thread function in a separate thread
threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

# Run the kohya_gui.py script and redirect its output to both the terminal and a file
with subprocess.Popen(["python", "kohya_gui.py", "--headless"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True) as process:
    for line in process.stdout:
        print(line, end='')  # Print output to terminal
        with open('ss.log', 'a') as f:
            f.write(line)  # Write output to file


In [ ]:
###@title # Zipfile to download dataset
zip_file_path = "" # @param {type:"string"}
extracted_folder_path = "" # @param {type:"string"}



import zipfile
import os


# Create a directory to extract the contents of the zip file
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print("Extraction complete.")
